In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
from IPython.display import display, HTML

sns.set(palette="bright", style="whitegrid")

# Loading datasets

Automatically checs for `.res` and `.prof` -files in the current directory.

In [ ]:
def machine_processor(path):
    if "CPU" in path:
        return path.split("_CPU")[0]
    if "Core" in path:
        return "_".join(path.split("-Core")[0].split("_")[:-1])
    if "avx" in path:
        return path.split("_avx")[0]
    return ".".join(path.split(".")[:-1])

In [ ]:
def mk_df(path):
    reg = re.compile(r"BM_([^\d]*?)(u?int\d+_t)(u?int\d+_t)(\d+)")
    cols = ["Benchmark", "Time (ns)", "u1", "CPU (ns)", "u2", "Iterations", "Checksum"]
    df = pd.read_csv(path, sep=r"\s+", skiprows=3, names=cols, index_col=0)
    df.drop(columns=["u1", "u2"], inplace=True)
    typ, dtype, itype, size = zip(*[reg.match(u).groups() for u in df.index])
    df["Type"] = typ
    df["Data type"] = dtype
    df["Index type"] = itype
    df["Size"] = [int(s) for s in size]
    df["Time (ns)"] = df["Time (ns)"] / 100000
    df["CPU (ns)"] = df["CPU (ns)"] / 100000
    df["Machine"] = machine_processor(path)
    df["Throughput"] = "avx" in path
    return df

In [ ]:
def mk_pdf(path):
    styps = {"Type", "Data type", "Index type", "Size"}
    rows = []
    with open(path) as in_file:
        entries = dict()
        for line in in_file:
            s_l = line.strip()
            if s_l.startswith("=="):
                if len(entries) > 0:
                    rows.append({k: v for k, v in entries.items()})
                    entries.clear()
                entries["Machine"] = machine_processor(path)
            elif len(s_l) > 0:
                s_l = s_l.split(":")
                if s_l[0] in styps:
                    entries[s_l[0]] = s_l[1].strip()
                else:
                    entries[s_l[0]] = int(s_l[1].strip()) if s_l[0].startswith("Checksum") else float(s_l[1].strip())
    tdf = pd.DataFrame(rows)
    tdf = tdf.astype({"Size": int})
    tdf["Throughput"] = "avx" in path
    return tdf

In [ ]:
bdf = pd.concat([
    mk_df(p)   
    for p in [f for f in os.listdir() if f.endswith("res")]
])
bdf.drop("Checksum", axis=1, inplace=True)

In [ ]:
pdf = pd.concat([
    mk_pdf(p)
    for p in [f for f in os.listdir() if f.endswith("prof")]
])

In [ ]:
df = pd.merge(bdf, pdf, "left", on=["Type", "Data type", "Index type", "Size", "Machine", "Throughput"])
df["Queries per second"] = 1000000000 / df["Time (ns)"]

In [ ]:
df.head()

In [ ]:
df.to_csv("results.csv")

# Check that checksums agree

All different methods should agree on the query results.

In [ ]:
for g in df.groupby(["Data type", "Size", "Throughput"]):
    cs = g[1]["Checksum"].dropna().unique()
    if len(cs) > 1:
        print(g[0])
        print(cs)
        print()

# Overview of (non thgroughput) performance

If a particular data type is significantly different in performance on a specific machine, this should show up here.

Generally it looks like 
* binary search is very noisy
* smaller data types are faster

In [ ]:
col_count = len(df["Machine"].unique())
row_count = len(df["Type"].unique())
styles = {
    d_typ : {"color": color, "label": d_typ}
    for d_typ, color in zip(df["Data type"].unique(), sns.color_palette())
}
plt.figure(figsize=(24, 40))
for c, (machine, m_df) in enumerate(df[df["Throughput"] == False].groupby("Machine")):
    for r, (typ, t_df) in enumerate(m_df.groupby("Type")):
        plt.subplot(row_count, col_count, 1 + r * col_count + c)
        for dtype, d_df in t_df.groupby("Data type"):
            plt.plot(d_df["Size"], d_df["Time (ns)"], **styles[dtype])
        plt.legend()
        if r == 0:
            plt.title(f"{machine}")
        if c == 0:
            plt.ylabel(f"{typ}\nTime (ns)")
        if r == row_count - 1:
            plt.xlabel("Array elements")
        plt.xscale("log")
plt.show()

# Plot by machine

To get performance overview on different architectures for different search strategies.

In [ ]:
type_styles = {
    typ: {"color": color, "label": typ}
    for typ, color in zip(df["Type"].unique(), sns.color_palette())
}

In [ ]:
def machine_comp(d_type="uint64_t", types={'linear_scan_cmov', 'templated_binary', 'templated_cmov',}):
    columns = 2
    rows = (len(df["Machine"].unique()) + 1) // columns
    plt.figure(figsize=(16, 12))
    for i, (m, m_df) in enumerate(df[(df["Throughput"] == False) & (df["Data type"] == d_type)].groupby("Machine")):
        plt.subplot(rows, columns, i + 1)
        for typ, t_df in m_df.groupby("Type"):
            if typ not in types:
                continue
            plt.plot(t_df["Size"], t_df["Time (ns)"], **type_styles[typ])
        plt.ylabel("Mean query time")
        plt.xlabel("Array size (elements)")
        plt.xscale("log")
        ticks = sorted(np.unique(g[1]["Size"]))
        plt.xticks(ticks=ticks, labels=ticks)
        plt.legend()
        plt.title(m)
        plt.ylim((0, 100))
    plt.show()

In [ ]:
machine_comp("int8_t", set(df["Type"].unique()))

In [ ]:
machine_comp("uint64_t", set(df["Type"].unique()))

# Comparison of $\mathcal(O)\log n$ strategies

In [ ]:
machine_comp("uint64_t", {"binary", "branchless_cmov", "templated_binary", "templated_cmov"})

# Comparisons by data type and architecture

Check what is actually fastest on which machine

In [ ]:
machine_comp("uint64_t")

In [ ]:
machine_comp("uint32_t")

In [ ]:
machine_comp("int16_t")

In [ ]:
machine_comp("uint8_t")

# Map profiling data to performance

Plot profiling data. Should explain the observed query performance.

In [ ]:
def plot_profiling(machine, d_type, save_pdf=False):
    typs = {
        'linear_scan_cmov',
        'templated_binary', 
        'templated_cmov'
    }
    plt.figure(figsize=(18, 18))
    tdf = df[(df["Machine"] == machine) & (df["Data type"] == d_type) & (df["Throughput"] == False)]
    ticks = sorted(tdf["Size"].unique())
    for i, col in enumerate(["Time (ns)", "L1D misses", "Instructions", "IPC", "Branch instructions", "Branch misspredictions"]):
        plt.subplot(3, 2, i + 1)
        maxiums = []
        for g in tdf.groupby("Type"):
            if g[0] not in typs:
                continue
            maxiums.append(g[1][col].max())
            plt.plot(g[1]["Size"], g[1][col], label=g[0])
        plt.xscale("log")
        plt.ylabel(col)
        plt.xticks(ticks=ticks, labels=ticks)
        plt.legend()
        if i > 1:
            plt.xlabel("Array size (elements)")
        if col not in {"IPC", "Branch misspredictions"}:
            maxiums.sort()
            plt.ylim((-0.5, int(maxiums[1]) + 1))
    plt.suptitle(f"{machine} - {d_type}")
    plt.savefig(f"{machine}_{d_type}.pdf", bbox_inches='tight')
    plt.show()

In [ ]:
plot_profiling("Intel_Core_i5-8250U", "uint64_t")

In [ ]:
plot_profiling("Intel_Xeon_Gold_6248", "uint64_t")

In [ ]:
plot_profiling("AMD_EPYC_7302", "uint64_t")

In [ ]:
plot_profiling("Cortex-A76", "uint64_t")

# Compare latency/throughput

Get some indicator of the effect of intra-query data dependece injection.

No data dependence, allows compile-time and run-time parallellization of the queries.

The high throughput approach, is likely not very usefull in practical applications.

In [ ]:
sns.relplot(
    data=df[(df["Type"] == "branchless_cmov") & (df["Data type"] == "uint64_t")], x="Size", y="Queries per second", hue="Throughput", 
    col="Machine", col_wrap=2, kind="line", facet_kws={"sharey": False}).set(xscale="log", yscale="log")
plt.show()